# Interfaces/calculators

## ASE Calculator: Equation of state (EOS)

First, do a bulk calculation for different lattice constants:



In [ ]:
import numpy as np

from ase import Atoms
from ase.io.trajectory import Trajectory
from ase.calculators.emt import EMT

a = 4.0  # approximate lattice constant
b = a / 2
ag = Atoms('Ag',
           cell=[(0, b, b), (b, 0, b), (b, b, 0)],
           pbc=1,
           calculator=EMT())  # use EMT potential
cell = ag.get_cell()

# Extract volumes and energies:
volumes = []
energies = []

for x in np.linspace(0.95, 1.05, 5):
    ag.set_cell(cell * x, scale_atoms=True)
    energies.append(ag.get_potential_energy())
    volumes.append(ag.get_volume())


This will write a trajectory file containing five configurations of FCC silver for five different lattice constants. Now, analyse the result with the `EquationOfState` class and this script:

In [ ]:
from ase.units import kJ
from ase.eos import EquationOfState

# configs = read('Ag.traj@0:5')  # read 5 configurations

eos = EquationOfState(volumes, energies)
v0, e0, B = eos.fit()
print(B / kJ * 1.0e24, 'GPa')
eos.plot()

## Generating input for Gaussian (or run it locally if available)

Defining all of the helper functions

In [ ]:
from pathlib import Path
from inspect import cleandoc

def generate_calc_raman(atoms,inp_file, method='B3LYP', basis='3-21G', nprocs=16, mem=7500, ):
    """Input file for the Gaussian calculation
    
    Args:
        atoms (Atoms): ASE Atoms object of the structure
        method (str): Name of the method (defualt: 'B3LYP')
        basis (str): Name of the basis set (defualt:'3-21G')
        nprocs (int): Number of cores used on the node
        mem (int): Memory in MB per CPU core
    """
    
    template = cleandoc(
        """
        %NProcShared={nprocs}
        %mem={mem}MB
        %chk={jobname}.chk
        #P {method}/{basis} OPT(CalcAll) FREQ(Raman)
        
        {name}
        
        0 1
        {structure}
        """)
    
    structure = '\n'.join('{} {} {} {}'.format(atom.symbol, *atom.position) for atom in atoms)
    
    return template.format(
        method=method,
        basis=basis,
        name=atoms.get_chemical_formula(),
        structure=structure,
        nprocs=nprocs,
        mem=nprocs*mem,
        jobname=inp_file.stem
    )
    

def generate_job_script(inp_file, nprocs=16, mem=7500, time=24):
    """Job script for the hercules node for gaussian calculations

    Args:
        nprocs (int): Number of cores used on the node
        mem (int): Memory in MB per CPU core
        time (int): The walltime in hours
    """
    
    template = cleandoc(
        """
        #!/bin/bash
        
        #SBATCH --job-name={jobname}
        #SBATCH --time={time}:00:00       # hh:mm:ss
        #SBATCH --nodes=1                 # Number of nodes to use
        #SBATCH --ntasks-per-node=1       # Number of tasks per node (aka MPI processes)
        #SBATCH --cpus-per-task={nprocs}  # Number of cpus per task (aka OpenMP threads)
        #SBATCH --mem-per-cpu={mem}       # megabytes
        
        module load Gaussian/16_A03-PGI-16.5
        
        echo "Node: $(hostname)"
        echo "Gaussian Path: $(which g16)"

        export GAUSS_SCRDIR="$TMPDIR"
        
        g16 {inp_file} 
        """)
    
    return template.format(
        jobname=inp_file.stem,
        time=time,
        nprocs=nprocs,
        mem=mem,
        inp_file=inp_file
    )
    

def construct_filename(filename, method, basis):
    
    for char in ['(', ')', ',']:
        basis = basis.replace(char, '')


    return filename.stem + '_' + method + '_' + basis

Structure and other parameters

In [ ]:
from ase.io import read


structfile = Path('data/benzene.xyz')
atoms = read(structfile)

folder = Path('gaussian/')

methods = ['B3LYP', 'PBEPBE', "MP2"]
basis_sets = ['3-21G', '6-31G(d)', '6-311+G(2d,p)']

nprocs = 1
mem = 3500 # MB per CPU core
walltime = 1 # hours


Generating input files

In [ ]:
for method in methods:
    for basis in basis_sets:
        
        basename = Path(construct_filename(structfile, method, basis))
        
        print(basename)

        with open(folder / basename.with_suffix('.inp'), 'w') as f:
            f.write(generate_calc_raman(atoms,basename.with_suffix('.inp'), method, basis, nprocs, mem))
            f.write('\n\n')

        with open(folder / basename.with_suffix('.sh'), 'w') as f:
            f.write(generate_job_script(basename.with_suffix('.inp'), nprocs, mem, walltime))
            f.write('\n\n')

## Using pymatgen

In [ ]:
from pymatgen.io.xyz import XYZ
from pymatgen.io.gaussian import GaussianInput

In [ ]:
structure = XYZ.from_file('data/benzene.xyz').molecule
inp = GaussianInput(
    structure,
    title='benzene',
    functional='B3LYP',
    basis_set='6-311+G(2d,p)',
    route_parameters={'OPT':'CalcAll', 'FREQ':'Raman'},
    input_parameters=None,
    link0_parameters={
        '%NProcShared': 1,
        '%mem':'3500MB',
        '%chk':'benzene_B3LYP_6-311+G2dp.chk'
    }
)

print(inp.to_string(cart_coords=True))